<br>
<font size='12px'>
Boosting Tree
</font>
<br>

[©2020 AI在弦上](https://www.dataml.cn)

# 概要

1. Boosting Tree 基本思想
   - 加模型, 基模型的线性组合
   - 基模型, 决策树
       - 分类问题, 2叉分类树
       - 回归问题, 2叉回归树

2. 回归 Boosting Tree 实现步骤
   1. 初始化残差
   2. 确定当前残差下, 最佳基模型
     - 遍历不同切分点,  确定对应的叶子节点取值
     - 计算残差, 平方误差
     - 确定最佳切分点, 及相应残差、平方和误差

   3. 最终模型的线性组合
   
注:

- 提升树被认为是**统计学习**中性能**最好**的方法之一
- 决策树桩(decision stump), 由一个根节点直接连接两个叶节点的简单决策树

## 时间成本
- 阅读 10 分钟
- 实践 30 分钟

## 参考

1. 统计学习方法, 李航, 第8章 提升方法, p137


# 目标

我们的目标通过构造过多个弱决策树桩， 就能对训练数据进行预测, 使其预测误差控制在一定范围内

模型的一般形式, 如下:

$$
f_M(x) = \displaystyle\sum_{m=1}^M{T(x, \Theta_m)}
$$

# 实现

## 环境

In [1]:
import numpy as np

## 数据

| 序号 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| x | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 |
| y | 5.56 | 5.70 | 5.91 | 6.40 | 6.80 | 7.05 | 8.90 | 8.70 | 9.00 | 9.05 |

In [2]:
x = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
y = np.array([5.56, 5.70, 5.91, 6.40, 6.80, 7.05, 8.90, 8.70, 9.00, 9.05])

## 方法

### 基模型

这里使用2叉树作为基模型, 其一般形式, 定义如下

In [3]:
def decision_stump(x, s, c1, c2):
    
    """
    切分点左侧值为c1, 右侧为c2
    
    Parameters
    ==========
    x, 输入
    s, 树桩切分点
    c1, 左侧节点取值
    c2, 右侧切点取值
       
    Return
    ======
    返回对应叶子节点取值
    
    """
 
    return np.where(x < s, c1, c2)

In [4]:
decision_stump(x, 2.5, 1, 3)

array([1, 1, 3, 3, 3, 3, 3, 3, 3, 3])

### 切分点

In [5]:
def gen_cut_points(x):
        
    """
    生成切分点
    
    Parameters
    ==========
    x, 训练样本输入
       
    Return
    ======
    相邻输入样本的中间点列表
    
    """
    
    points = np.unique(x)
    return (points[:-1] + points[1:]) / 2

In [6]:
gen_cut_points(x)

array([1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, 9.5])

### 平方和误差函数

In [7]:
def calc_sse(x, y, m):
    
    """
    计算加权数据集错误率
    
    Parameters
    ==========
    x, 训练样本输入
    y, 训练样本标签
    m, 回归基模型
    
    Return
    ======    
    sse, 平方和误差(sum square error)
    residual, 残差
    
    """
    
    residual = y - m(x)
    sse = np.sum(residual ** 2)

    return sse, residual

### 最佳叶子节点取值

In [8]:
def calc_best_leaf_value(x, y, s):
    return np.mean(y[x < s]), np.mean(y[x > s])

In [9]:
calc_best_leaf_value(x, y, 2.5)

(5.63, 7.72625)

### 生成树桩函数

In [10]:
def gen_decision_stump(x, y, s):
    c1, c2 = calc_best_leaf_value(x, y, s)
    return lambda x: decision_stump(x, s, c1, c2)

### 最小误差树桩

In [11]:
def find_decision_stump(x, y, cut_points=None):
    
    """
    给定残差, 寻找最小平方和误差对应的树桩, 即对应的切分点及左、右叶子节点取值
    
    Parameters
    ==========
    x, 训练样本输入
    y, 训练样本标签
    cut_points, 切分点
    
    Return
    ======    
    cut_point, 切分点
    sse, 平方和误差
    residual, 残差
    basic_model, 函数
    
    """
        
    cut_points = gen_cut_points(x) if cut_points is None else cut_points
        
    error_info = [calc_sse(x, y, gen_decision_stump(x, y, c)) for c in cut_points]
    errors = np.array([e[0] for e in error_info])
    residuals = np.array([e[1] for e in error_info])
    
    min_index = np.argmin(errors)
    cut_point, sse, residual = cut_points[min_index], errors[min_index], residuals[min_index]
    
    return cut_point, sse, residual, gen_decision_stump(x, y, cut_point)

In [12]:
cut_point, sse, residual, m = find_decision_stump(x, y)
cut_point, sse, residual, m

(6.5,
 1.9300083333333335,
 array([-0.67666667, -0.53666667, -0.32666667,  0.16333333,  0.56333333,
         0.81333333, -0.0125    , -0.2125    ,  0.0875    ,  0.1375    ]),
 <function __main__.gen_decision_stump.<locals>.<lambda>>)

### 模型融合

In [13]:
def gen_additive_model(models):
    return lambda x: np.sum([m(x) for m in models], axis=0)

### 训练模型

In [14]:
def boosting_tree(x, y, M=10, max_sse=0):
  
    cut_points = gen_cut_points(x)
    residual = np.copy(y)
    
    models = list()
    
    for m in range(M):
        
        cut_point, sse, residual, tree = find_decision_stump(x, residual, cut_points=cut_points)
        models.append(tree)
        
        print('\niterator: ', m + 1) 
        print('cut point: ', cut_point)
        print('sse: ', sse)
        print('residual: ', residual)

        if sse <= max_sse:
            print('find the target model')
            break
            
    model = gen_additive_model(models)
    return model


In [15]:
model = boosting_tree(x, y, max_sse=0.2)


iterator:  1
cut point:  6.5
sse:  1.9300083333333335
residual:  [-0.67666667 -0.53666667 -0.32666667  0.16333333  0.56333333  0.81333333
 -0.0125     -0.2125      0.0875      0.1375    ]

iterator:  2
cut point:  3.5
sse:  0.8006750000000017
residual:  [-0.16333333 -0.02333333  0.18666667 -0.05666667  0.34333333  0.59333333
 -0.2325     -0.4325     -0.1325     -0.0825    ]

iterator:  3
cut point:  6.5
sse:  0.478008333333334
residual:  [-0.31       -0.17        0.04       -0.20333333  0.19666667  0.44666667
 -0.0125     -0.2125      0.0875      0.1375    ]

iterator:  4
cut point:  4.5
sse:  0.3055592592592598
residual:  [-0.14916667 -0.00916667  0.20083333 -0.0425      0.08944444  0.33944444
 -0.11972222 -0.31972222 -0.01972222  0.03027778]

iterator:  5
cut point:  6.5
sse:  0.22891522633744912
residual:  [-0.22064815 -0.08064815  0.12935185 -0.11398148  0.01796296  0.26796296
 -0.0125     -0.2125      0.0875      0.1375    ]

iterator:  6
cut point:  2.5
sse:  0.17217806498628296

In [16]:
model(x)

array([5.63      , 5.63      , 5.81831019, 6.55164352, 6.81969907,
       6.81969907, 8.95016204, 8.95016204, 8.95016204, 8.95016204])